### Load Networks
- loads each trained network into memory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from banditpy.models import BanditTrainer2Arm
from pathlib import Path

basepath = Path("D:/Data/mab/rnn_models/probs_1decimals")
struc_models, unstruc_models = [], []

for i in range(10):
    # ------ Structured network ----------
    s = BanditTrainer2Arm(model_path=basepath / f"structured_2arm_model{i}.pt")
    s.load_model()
    struc_models.append(s)

    # ------ Untructured network ----------
    u = BanditTrainer2Arm(model_path=basepath / f"unstructured_2arm_model{i}.pt")
    u.load_model()
    unstruc_models.append(u)

### Load and test Network
Using the loaded model/network it creates test sessions for each network that are saved as csv files.

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
from banditpy.models import BanditTrainer2Arm
from banditpy.utils import generate_probs_2arm

n_sessions = 1000
modelpath = Path("D:/Data/mab/rnn_models/probs_1decimals_0.16impure")
datapath = Path("D:/Data/mab/rnn_data/trained_1decimals_0.16impure")

# prob_kwargs = dict(high=0.91, low=0.1, decimals=1)
probs = np.array([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
unstruc_probs, struc_probs = generate_probs_2arm(
    probs, N=n_sessions, frac_impurity=0.16
)

for parent in ["structured_2arm_model", "unstructured_2arm_model"]:
    for i in range(10):
        s = BanditTrainer2Arm(model_path=modelpath / f"{parent}{i}.pt")
        s.load_model()
        model_name = s.model_path.stem
        model_folder = datapath / model_name
        model_folder.mkdir(exist_ok=True)

        # ------ Structured testing ----------
        struc_exp_folder = model_folder / f"{model_name}_structured"
        struc_exp_folder.mkdir(exist_ok=True)
        dfs = s.evaluate(mode=struc_probs, n_sessions=n_sessions)
        dfs.to_csv(struc_exp_folder / f"{model_name}_structured.csv", index=False)

        # ------ Unstructured testing ----------
        unstruc_exp_folder = model_folder / f"{model_name}_unstructured"
        unstruc_exp_folder.mkdir(exist_ok=True)
        dfu = s.evaluate(mode=unstruc_probs, n_sessions=n_sessions)
        dfu.to_csv(unstruc_exp_folder / f"{model_name}_unstructured.csv", index=False)


# for models in [struc_models, unstruc_models]:
#     for i, model in enumerate(models):
#         model_name = model.model_path.stem
#         model_folder = basepath / model_name
#         model_folder.mkdir(exist_ok=True)

#         struc_exp_folder = model_folder / f"{model_name}_structured"
#         struc_exp_folder.mkdir(exist_ok=True)
#         dfs = model.evaluate(mode="S", n_sessions=300, **prob_kwargs)
#         dfs.to_csv(struc_exp_folder / f"{model_name}_structured.csv", index=False)

#         unstruc_exp_folder = model_folder / f"{model_name}_unstructured"
#         unstruc_exp_folder.mkdir(exist_ok=True)
#         dfu = model.evaluate(mode="U", n_sessions=300, **prob_kwargs)
#         dfu.to_csv(unstruc_exp_folder / f"{model_name}_unstructured.csv", index=False)

### Plotting loss history and network trajectory

In [26]:
from neuropy import plotting
from mab_colors import colors_2arm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pathlib import Path
import mab_subjects

fig = plotting.Fig(5, 7, num=1, size=(14, 11), fontsize=10)

modelpath = Path("D:/Data/mab/rnn_models/probs_1decimals")
unstruc_model = BanditTrainer2Arm(model_path=modelpath / f"unstructured_2arm_model3.pt")
unstruc_model.load_model()

struc_model = BanditTrainer2Arm(model_path=modelpath / f"structured_2arm_model1.pt")
struc_model.load_model()

loss_history = [unstruc_model.training_loss_history, struc_model.training_loss_history]


for i, loss in enumerate(loss_history):
    ax = fig.subplot(fig.gs[i, 0])
    ax.plot(loss, colors_2arm()[i])
    ax.set_title(["Unstructured", "Structured"][i])
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Log loss")


u_on_u = mab_subjects.rnn_u_on_u[0].b2a
u_on_s = mab_subjects.rnn_u_on_s[0].b2a

ax = fig.subplot(fig.gs[0, 1])
for i, task in enumerate([u_on_u, u_on_s]):
    ax.plot(task.get_optimal_choice_probability(), colors_2arm([1, 1.3][i])[0])
    ax.set_xlabel("Trials")
    ax.set_ylim(0.4, 1)
    ax.set_ylabel("Pr (High)")
ax.legend(["self", "struc"], loc="lower right")
ax.set_title("Unstructured network")

s_on_s = mab_subjects.rnn_s_on_s[0].b2a
s_on_u = mab_subjects.rnn_s_on_u[0].b2a

ax = fig.subplot(fig.gs[1, 1])
for i, task in enumerate([s_on_s, s_on_u]):
    ax.plot(task.get_optimal_choice_probability(), colors_2arm([1, 1.4][i])[1])
    ax.set_xlabel("Trials")
    ax.set_ylim(0.4, 1)
    ax.set_ylabel("Pr (High)")
ax.legend(["self", "unstruc"], loc="lower right")
ax.set_title("Structured network")

cmap = sns.color_palette("hot", as_cmap=True)
# cmap = sns.cubehelix_palette(as_cmap=True)

# ----- unstruc model ------
rw = [
    [[0.8, 0.1], [0.1, 0.8]],
    [[0.8, 0.2], [0.2, 0.8]],
]

for m, model in enumerate([unstruc_model, struc_model]):
    data0 = model.analyze_hidden_states(reward_probs=rw[m][0], n_trials=100)
    data1 = model.analyze_hidden_states(reward_probs=rw[m][1], n_trials=100)

    hs0 = np.array(data0["hidden_states"])  # Shape: (n_trials, hidden_size)
    hs1 = np.array(data1["hidden_states"])  # Shape: (n_trials, hidden_size)

    actions0 = np.array(data0["actions"])  # Shape: (n_trials,)
    actions1 = np.array(data1["actions"])  # Shape: (n_trials,)

    hidden_states = np.concatenate(
        [hs0, hs1], axis=0
    )  # Shape: (2*n_trials, hidden_size)
    actions = np.concatenate([actions0, actions1], axis=0)  # Shape: (2*n_trials,)

    marker_size = np.ones_like(actions) * 10
    # marker_size[actions == np.argmax(rw) + 1] = 10  # Size for high arm
    # marker_size[actions == np.argmin(rw) + 1] = 30  # Size for low arm

    # Apply PCA to reduce to 2D
    pca = PCA(n_components=2)
    hidden_states_2d = pca.fit_transform(hidden_states)

    # 1. PCA trajectory colored by time
    ax = fig.subplot(fig.gs[m, 2])
    scatter1 = ax.scatter(
        hidden_states_2d[: len(actions0), 0],
        hidden_states_2d[: len(actions0), 1],
        c=range(100),
        cmap=cmap,
        s=5,
    )

    ax.plot(
        hidden_states_2d[:100, 0],
        hidden_states_2d[:100, 1],
        "k-",
        alpha=0.3,
        linewidth=1,
    )
    ax.set_xlim(-3, 3)
    # ax.set_ylim(0, 3)

    # 1. PCA trajectory colored by time
    ax = fig.subplot(fig.gs[m, 3])
    scatter2 = ax.scatter(
        hidden_states_2d[100:200, 0],
        hidden_states_2d[100:200, 1],
        c=range(100),
        cmap=cmap,
        s=5,
    )

    ax.plot(
        hidden_states_2d[100:200, 0],
        hidden_states_2d[100:200, 1],
        "k-",
        alpha=0.3,
        linewidth=1,
    )
    ax.set_xlim(-3, 3)
    # ax.set_ylim(0, 3)


# for r, rw in enumerate(reward_probs):
#     for i, rnn in enumerate([unstruc_model, struc_model]):
#         ax = fig.subplot(fig.gs[i, r + 2])

#         session_data = rnn.analyze_hidden_states(reward_probs=rw, n_trials=100)
#         hidden_states = np.array(
#             session_data["hidden_states"]
#         )  # Shape: (n_trials, hidden_size)
#         actions = np.array(session_data["actions"])  # Shape: (n_trials,)
#         marker_size = np.ones_like(actions)

#         # marker_size[actions == 1] = 5  # Size for arm 1
#         # marker_size[actions == 2] = 15  # Size for arm 2

#         marker_size[actions == np.argmax(rw) + 1] = 10  # Size for high arm
#         marker_size[actions == np.argmin(rw) + 1] = 30  # Size for low arm

#         # Apply PCA to reduce to 2D
#         pca = PCA(n_components=2)
#         hidden_states_2d = pca.fit_transform(hidden_states)

#         # 1. PCA trajectory colored by time
#         scatter1 = ax.scatter(
#             hidden_states_2d[:, 0],
#             hidden_states_2d[:, 1],
#             c=range(len(hidden_states_2d)),
#             cmap=cmap,
#             s=marker_size,
#         )
#         ax.plot(
#             hidden_states_2d[:, 0], hidden_states_2d[:, 1], "k-", alpha=0.3, linewidth=1
#         )

#         # ax.set_title(
#         #     f"Trajectory (PCA)\nVar explained: {pca.explained_variance_ratio_.sum():.3f}"
#         # )

#         ax.set_title(f"Trajectory (PCA)\nreward: {rw}")

#         ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.3f})")
#         ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.3f})")

#         # if r == 0:
#         #     plt.colorbar(scatter1, ax=ax, label="Trial")

Model and training history loaded from D:\Data\mab\rnn_models\probs_1decimals\unstructured_2arm_model3.pt
Loaded 30000 training loss values
Model and training history loaded from D:\Data\mab\rnn_models\probs_1decimals\structured_2arm_model1.pt
Loaded 30000 training loss values


In [4]:
u_on_u

MABData(unstructured_2arm_model0_unstructured)